In [3]:
# setup
from IPython.core.display import display,HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
display(HTML(open('rise.css').read()))

# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})


# CMPS 2200
# Introduction to Algorithms

## Edit Distance, Longest Increasing Subsequence


Today's agenda:

- Two dynamic programming algorithms for Edit Distance and Longest Increasing Subsequence

### Edit Distance

Given two strings $S, T \in \Sigma^*$, how similar are they?

We can measure this using *edit distance*, which is the number of insertions and deletions needed to turn $S$ into $T$. Note that we can also go from $T$ to $S$ if we just reverse the edits (by turning insertions into deletions)

Example: $S$ = `abcdefghijkl`, $T$ = `abcdghikjl`. How many edits are needed?

Consider following edit sequence:

$S$: `abcdefghijkl---`<br>
$T$: `abcd--ghi---kjl`

This has 5 deletions and 3 insertions, for a total of 8 edits. What about this one:

$S$: `abcdefghijk-l`<br>
$T$: `abcd--ghi-kjl`

We have 3 deletions and 1 insertion for a total of 4 edits.

Our goal is to compute the **minimum edit distance** between two strings $S$ and $T$ of lengths $m$ and $n$, respectively.

It might seem like a toy problem, but this is a critical problem in comparing gene and protein sequences. By attaching  weights to insertions and deletions, we can assess the evolutionary distance between two sequences.

### Edit Distance

Notice that once again, if we greedily apply edits to the beginning or end of the string we might miss a set of edits interspersed throughout the string. 

Can we identify an optimal substructure propety for this problem?

Let's use case-based reasoning about the optimal solution as we did for Knapsack. Let the optimal number of edits between $S$ and $T$ be denoted $\mathit{MED}(S, T)$. In an optimal sequence of edits, how would we deal with the first two characters of $S$ and $T$, respectively?

For the base cases, if either string is empty, then the edit cost is simply the length of the other string.

If $S[0] = T[0]$, then there is no benefit to editing and $\mathit{MED}(S, T) = \mathit{MED}(S[1:], T[1:])$. However if $S[0] \neq T[0]$, then we must incur 1 edit. This edit is either an insertion into $S$ or a deletion in $S$, and we can decide which is preferable by considering the resulting subproblems. Thus we must evaluate $\mathit{MED}(S[1:], T)$ (delete $S[0]$) and $\mathit{MED}(S, T[1:) (insert at front of $S$).

**Optimal Substructure for Edit Distance**: Let $S$ and $T$ be strings of length $m$ and $n$. Then,

$$\mathit{MED}(S, T) = 
\begin{cases}
\mathit{MED}(S, T), \mbox{if}~~~S[0]=T[0] \\
1+\min\{\mathit{MED}(S[1:], T),\mathit{MED}(S, T[1:])\}, \mbox{otherwise} \\
\end{cases}
$$

Just as with Knapsack, this recursion tree for this recurrence yields an exponential number of nodes. How many nodes are there, and what is the depth? 


The recursion tree has $O(2^{m+n})$ nodes and depth $O(m+n)$. Are there shared subproblems?

For $S$=`ABC` and $T$=`DBC` we have the following DAG:

<img src = "edit_distance_DAG.jpg" width = "60%">

How much sharing is possible? In other words, how many distinct subproblems are there?

In any recursive call, the subproblems we consider consist of strings with one less character. So there are $O(mn)$ subproblems, which can each be computed in $O(1)$ time (if we have precomputed the necessary dependencies). The longest path in the recursion DAG is $O(m+n)$.


In [21]:
def MED(S, T):
    #print("S:%s, T:%s" % (S, T))
    if (S == ""):
        return(len(T))
    elif (T == ""):
        return(len(S))
    else:
        if (S[0] == T[0]):
            return(MED(S[1:], T[1:]))
        else:
            return(1 + min(MED(S, T[1:]), MED(S[1:], T)))

S= "back"
T= "baok"
print(MED(S, T))


2


## Longest Increasing Subsequence

We previously looked at approaches to identify trends in a sequence: longest run, maximum contiguous subsequence, longest gap. Let's look another trend we might want to identify from a sequence. Given a sequence $S = \langle s_0, s_1, \ldots, s_{n-1} \rangle$, what is the longest increasing subsequence? Note that subsequences don't need to be contiguous.

Example: $S=\langle 5, 2, 8, 6, 3, 6, 9, 7\rangle$. Every subsequence of length 1 is trivially increasing. Also $\langle 2, 6, 9 \rangle$, $\langle 2, 8, 9 \rangle$ are increasing, as is $\langle 5, 6, 7\rangle$. What is the longest?



Let's reduce this problem to something slightly simpler with the observation that the longest increasing subsequence must start somewhere in $S$.

Let $\mathit{LIS}(S, i)$ be the longest increasing subsequence for $S$ that starts with $S[i]$ as the first element. If $S[i]$ is the first element, then the next element $j$ must have $j>i$ and $S[j] > S[i]$. Whichever element is next, we must have $\mathit{LIS}(S, i) = 1 + \max_{j: S[j] > S[i]} \mathit{LIS}(S[j:]).$

Notice that if we can compute $LIS(S, i)$ then we can compute $ \mathit{LIS}(S) = \max_{0\leq i < n} \mathit{LIS}(S, i).$

**Optimal Substructure for Longest Increasing Subsequence**: Given a sequence $S$, we have that $ \mathit{LIS}(S) = \max_{0\leq i < n} \mathit{LIS}(S, i)$ where
$$\mathit{LIS}(S, i) = 1 + \max_{j: S[j] > S[i]} \mathit{LIS}(S, j).$$

This optimal substructure property is little different than what we've seen so far. The work to compute it, even if we have computed all subproblems already, is actually linear in the size of the sequence we consider (instead of $O(1)$). However there are only a linear number of starting points for an optimal solution. 

To compute this optimal substructure property, how many distinct subproblems must be computed from scratch? 


This problem is a little different than the other ones we've seen so far. If $S$ has length $n$, then we may need to take the maximum over the results of $n$ different subproblems. However the number of distinct subproblems is simply the number of prefixes of $S$. 

Thus, we can do $O(n^2)$ work if we save reuse the results from already visited subproblems. Since we decrease the length of the list by at least one element in each recursive call, the span is $O(n)$.


In [52]:
# longest increasing subsequence starting at position 0
def LIS_helper(S):
    #print(S)
    if (S == [] or len(S) == 1):
        return(len(S))
    else:
        # find elements in the sequence that are larger than S[0]
        rest = [j for j in range(len(S[1:])) if S[j]>S[0]]
        if (rest == []):
            return(1)
        else:
            results = [LIS_helper(S[i:]) for i in rest]
            return(1 + max(results))
    
def LIS(S):
    return(max([LIS_helper(L[i:]) for i in range(len(L))]))

L = [5, 2, 8, 6, 3, 6, 9, 7, 10]
print(LIS(L))

4
